In [1]:
%load_ext tensorboard

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers

from keras_visualizer import visualizer
from IPython.display import Image
import time
import datetime

ModuleNotFoundError: No module named 'keras_visualizer'

In [ ]:
tf.__version__

In [ ]:
### ignore TensorFlow INFO messages
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'

In [ ]:
HIDDEN_LAYER_1 = 8
HIDDEN_LAYER_2 = 0

model = tf.keras.Sequential()
model.add(layers.Dense(HIDDEN_LAYER_1, input_dim=2, activation='sigmoid'))

if HIDDEN_LAYER_2 > 0:
    model.add(layers.Dense(HIDDEN_LAYER_2, activation='sigmoid'))

model.add(layers.Dense(1, activation='sigmoid'))

In [ ]:
model.summary()

In [ ]:
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"), "%d_%d" % (HIDDEN_LAYER_1, HIDDEN_LAYER_2))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

In [ ]:
# dot muss im env/bin Verzeichnis sein, beim Starten des Jupyter Notebooks
visualizer(model, format='svg', view=False)
Image(url=f'graph.svg?{str(int(time.time()))}')

In [ ]:
model.compile(
  loss='binary_crossentropy', 
  optimizer=tf.keras.optimizers.SGD(learning_rate=1), 
  metrics=['accuracy'])

In [ ]:
training_data = np.array([[0,0], [0,1], [1,0], [1,1]])
target_data   = np.array([  [0],   [1],   [1],   [0]])

In [ ]:
epochs = 750
if HIDDEN_LAYER_2 > 0:
    epochs = 1500

t1 = time.time()
model.fit(training_data,
          target_data, 
          callbacks=[tensorboard_callback], 
          epochs=epochs)
t2 = time.time()

In [ ]:
print("time %f.3" % (t2 - t1))

In [ ]:
model.predict(training_data)

## Generate model.cpp

In [ ]:
from tinymlgen import port
import os

In [ ]:
path = "../pico-tflmicro/examples/xor"

In [ ]:
c_code = port(model, optimize=False, pretty_print = True)

open(path + "/model.cpp", "w").write('#include "model.h"\n' + c_code)

<pre>







</pre>
## Save tflite file and code generation

In [ ]:
### see https://github.com/eloquentarduino/tinymlgen

import re
import hexdump

def my_port(tflite_model, variable_name='model_data', pretty_print=True):
    bytes = hexdump.dump(tflite_model).split(' ')
    c_array = ', '.join(['0x%02x' % int(byte, 16) for byte in bytes])
    c = 'const unsigned char %s[] DATA_ALIGN_ATTRIBUTE = {%s};' % (variable_name, c_array)
    if pretty_print:
        c = c.replace('{', '{\n\t').replace('}', '\n}')
        c = re.sub(r'(0x..?, ){12}', lambda x: '%s\n\t' % x.group(0), c)
    c += '\nconst int %s_len = %d;' % (variable_name, len(bytes))
    
    preamble = '''
#include "model.h"

// if having troubles with min/max, uncomment the following
// #undef min    
// #undef max
#ifdef __has_attribute
#define HAVE_ATTRIBUTE(x) __has_attribute(x)
#else
#define HAVE_ATTRIBUTE(x) 0
#endif
#if HAVE_ATTRIBUTE(aligned) || (defined(__GNUC__) && !defined(__clang__))
#define DATA_ALIGN_ATTRIBUTE __attribute__((aligned(4)))
#else
#define DATA_ALIGN_ATTRIBUTE
#endif
'''
    return preamble + c


In [ ]:
def get_filename(optimizer):
    optimezedStr = "_optimized" if optimizer else ""
    return path + "/model_%d_%d%s.tflite" % (HIDDEN_LAYER_1, HIDDEN_LAYER_2, optimezedStr)

In [ ]:
def save_tflite(model, optimizer=True):
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    if optimizer:
        optimizers = [tf.lite.Optimize.DEFAULT]        
        converter.optimizations = optimizers
        
        def representative_dataset():
            data = training_data
            yield [data.astype(np.float32)]
        
        converter.representative_dataset = representative_dataset

    filename = get_filename(optimizer)

    tflite_model = converter.convert()
    print("%s %d bytes" % (filename, len(tflite_model)))

    with open(filename, 'wb') as f:
      f.write(tflite_model)
    
    code = my_port(tflite_model)
    open(filename + ".cpp", "w").write(code)

In [ ]:
save_tflite(model, optimizer=True)

In [ ]:
save_tflite(model, optimizer=False)

## Predict tflite files

In [ ]:
def predict_tflite(optimizer):
    filename = get_filename(optimizer)
    interpreter = tf.lite.Interpreter(model_path=filename)
    interpreter.allocate_tensors()

    input_index = interpreter.get_input_details()[0]["index"]
    output_index = interpreter.get_output_details()[0]["index"]

    data = [[tf.cast(0.0, tf.float32), tf.cast(0.0, tf.float32)]]

    interpreter.set_tensor(input_index, data)
    interpreter.invoke()
    pred = interpreter.get_tensor(output_index)
    print("%s %f" % (filename, pred))

    
predict_tflite(optimizer=False)
predict_tflite(optimizer=True)